In [ ]:
#!pip install gtfs_functions
#!pip install KeplerGl
#!pip install keplergl jupyter
#!jupyter nbextension install --py --symlink --sys-prefix keplergl
#!jupyter nbextension enable --py --sys-prefix keplergl
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension
#!pip install gtfs_functions matplotlib basemap
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import gtfs_functions as gtfs
from gtfs_functions import Feed
import networkx as nx
import geopandas as gpd
import folium
from shapely.geometry import Point, LineString
from folium.plugins import MarkerCluster
#import keplergl as kp
from google.colab import output
from IPython.display import display
import pandas as pd
from gtfs_functions.gtfs_plots import map_gdf
import plotly.graph_objs as go
import matplotlib.colors as mcolors

In [ ]:
feed = Feed("budapest_gtfs.zip", time_windows=[6, 7]) #itt változtathatunk az időintervallumon
#feed = Feed("budapest_gtfs.zip")
routes = feed.routes
trips = feed.trips
stops = feed.stops
stop_times = feed.stop_times
shapes = feed.shapes

In [ ]:
segments_gdf = feed.segments
segments_gdf.head(2)
#segments_gdf.info()

In [ ]:
segments_freq = feed.segments_freq
segments_freq.head(2)
#print(type(segments_freq))

In [ ]:
colnames=['route_name', 'capacity']
df = pd.read_csv('capacity.csv', names=colnames, header=None)
df.head(2)

In [ ]:
segments_freq = segments_freq.merge(df, on='route_name')
segments_freq.head(2)

In [ ]:
def add_edges(G, gdf):
    for idx, row in gdf.iterrows():
        # Extract start and end coordinates
        start_coords = (row['geometry'].coords[0][1], row['geometry'].coords[0][0])
        end_coords = (row['geometry'].coords[-1][1], row['geometry'].coords[-1][0])

        # Calculate the product of ntrips and capacity
        ntrips_capacity = row['ntrips'] * row['capacity']

        # Create a unique key for the edge
        key = f"{row['route_name']}_{ntrips_capacity}"

        # Add edge to the graph with specified keys and additional attributes
        G.add_edge(start_coords, end_coords, key=key, route_name=row['route_name'], ntrips_capacity=ntrips_capacity)

In [ ]:
G = nx.MultiDiGraph()
# Add edges from segments_freq to the graph
add_edges(G, segments_freq)

# Display basic information about the graph
#print(info(G))

In [ ]:
def create_folium_map(G):
    # Create a folium map centered around Budapest
    m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)

    # Add edges to the folium map with tooltips
    for u, v, data in G.edges(data=True):
        # Create the coordinates list for the PolyLine
        coords = [u, v]

        # Create the tooltip text
        tooltip_text = f"Route: {data['route_name']}<br>Capacity: {data['ntrips_capacity']}"

        # Add the PolyLine to the map with the tooltip
        folium.PolyLine(coords, color='blue', weight=2.5, opacity=1, tooltip=tooltip_text).add_to(m)

    # Add nodes to the folium map
    for node in G.nodes():
        folium.CircleMarker(location=node, radius=2, color='red').add_to(m)

    return m

In [ ]:
def create_folium_map(G):
    # Create a folium map centered around Budapest
    m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)

    # Generate a list of distinct colors
    colors = list(mcolors.CSS4_COLORS.values())
    max_colors = min(len(colors), 50)  # Limit to 30 distinct colors if there are fewer colors available
    colors = colors[:max_colors]

    # Create a dictionary to track the number of parallel edges
    edge_count = {}

    # Add edges to the folium map with tooltips
    for u, v, key, data in G.edges(keys=True, data=True):
        # Create the coordinates list for the PolyLine
        coords = [u, v]

        # Create a unique identifier for the edge pair (u, v)
        edge_id = (u, v)
        if edge_id not in edge_count:
            edge_count[edge_id] = 0
        edge_index = edge_count[edge_id]
        edge_count[edge_id] += 1

        # Assign a color based on the edge index
        color = colors[edge_index % len(colors)]

        # Create the tooltip text
        tooltip_text = f"Route: {data['route_name']}<br>Capacity: {data['ntrips_capacity']}"

        # Add the PolyLine to the map with the tooltip
        folium.PolyLine(coords, color=color, weight=2.5, opacity=1, tooltip=tooltip_text).add_to(m)

    # Add nodes to the folium map
    for node in G.nodes():
        folium.CircleMarker(location=node, radius=2, color='red').add_to(m)

    return m

In [ ]:
s = create_folium_map(G)

# Display the map
#m.save("multidigraph_map.html")
s

In [ ]:
edges_gdf = gpd.GeoDataFrame(columns=['geometry', 'route_name', 'ntrips'])
for u, v, data in G.edges(data=True):
    data['geometry'] = LineString([u, v])
    pd.concat([edges_gdf, gpd.GeoDataFrame([data])])

# Initialize KeplerGl map
m = kp.KeplerGl(height=400)

# Add data to KeplerGl map
m.add_data(data=edges_gdf, name='Segments')

# Configure map layers and tooltips
m.config = {
#config = {
    "version": "v1",
    "config": {
        "mapState": {
            "latitude": 47.4979,
            "longitude": 19.0402,
            "zoom": 12
        },
        "mapStyle": {
            "styleType": "dark"
        },
        "visState": {
            "layers": [
                {
                    "id": "Segments",
                    "type": "line",
                    "config": {
                        "dataId": "Segments",
                        "label": "Segments",
                        "columns": {
                            "geojson": "geometry"
                        },
                        "isVisible": True,
                        "visConfig": {
                            "opacity": 0.8,
                            "thickness": 1,
                            "colorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300"
                                ]
                            },
                            "strokeColor": [170, 255, 0],
                            "strokeWidth": 0.5
                        },
                        "textLabel": [
                            {
                                "field": {
                                    "name": "route_name"
                                },
                                "color": [255, 255, 255],
                                "size": 18,
                                "offset": [0, 0],
                                "anchor": "start",
                                "alignment": "center"
                            }
                        ]
                    }
                }
            ]
        }
    }
}

# Display the map (this would work in a Jupyter notebook or similar environment)
m
output.enable_custom_widget_manager()

In [ ]:
def add_edges_with_weights(G, gdf):
    for idx, row in gdf.iterrows():
        # Extract start and end coordinates
        start_coords = (row['geometry'].coords[0][1], row['geometry'].coords[0][0])
        end_coords = (row['geometry'].coords[-1][1], row['geometry'].coords[-1][0])

        ntrips_capacity = row['ntrips'] * row['capacity']

        # Add edge to the graph with ntrips as weight
        G.add_edge(start_coords, end_coords, weight=ntrips_capacity, attr_dict=row.to_dict())

In [ ]:
def add_edges_with_weights2(G, gdf):
    for idx, row in gdf.iterrows():
        # Extract start and end coordinates
        start_coords = (row['geometry'].coords[0][1], row['geometry'].coords[0][0])
        end_coords = (row['geometry'].coords[-1][1], row['geometry'].coords[-1][0])

        ntrips_capacity = row['ntrips'] * row['capacity']

        if G.has_edge(start_coords, end_coords):
            # If the edge exists, update the weight
            G[start_coords][end_coords]['weight'] += ntrips_capacity
            # Optionally, update the ntrips_capacity attribute to reflect the new total
            G[start_coords][end_coords]['ntrips_capacity'] += ntrips_capacity
        else:
            # If the edge does not exist, add it with the new weight and attributes
            G.add_edge(start_coords, end_coords, weight=ntrips_capacity, route_name=row['route_name'], ntrips_capacity=ntrips_capacity)

In [ ]:
G2 = nx.DiGraph()
add_edges_with_weights2(G2, segments_freq)

In [ ]:
def create_folium_map2(G):
    # Create a folium map centered around Budapest
    m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)

    # Add edges to the folium map with tooltips
    for u, v, data in G.edges(data=True):
        # Create the coordinates list for the PolyLine
        coords = [u, v]

        # Create the tooltip text
        capacity = data.get('ntrips_capacity', 'N/A')
        tooltip_text = f"Capacity: {capacity}"

        # Add the PolyLine to the map with the tooltip
        folium.PolyLine(coords, color='blue', weight=2.5, opacity=1, tooltip=tooltip_text).add_to(m)

    # Add nodes to the folium map
    for node in G.nodes():
        folium.CircleMarker(location=node, radius=2, color='red').add_to(m)

    return m

In [ ]:
r = create_folium_map2(G2)
r

In [ ]:
edges_data = []
for u, v, data in G.edges(data=True):
    edges_data.append({
        'geometry': LineString([u, v]),
        'route_name': data['route_name'],
        'ntrips': data['ntrips']
    })

edges_gdf = gpd.GeoDataFrame(edges_data, columns=['geometry', 'route_name', 'ntrips'])

# Initialize KeplerGl map
m = kp.KeplerGl(height=600)

# Add data to KeplerGl map
m.add_data(data=edges_gdf, name='Segments', replace=True)

# Configure map layers and tooltips
m.config = {
    "version": "v1",
    "config": {
        "mapState": {
            "latitude": 47.4979,
            "longitude": 19.0402,
            "zoom": 12
        },
        "mapStyle": {
            "styleType": "dark"
        },
        "visState": {
            "layers": [
                {
                    "id": "Segments",
                    "type": "line",
                    "config": {
                        "dataId": "Segments",
                        "label": "Segments",
                        "columns": {
                            "geojson": "geometry"
                        },
                        "isVisible": True,
                        "visConfig": {
                            "opacity": 0.8,
                            "thickness": 1,
                            "colorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300"
                                ]
                            },
                            "strokeColor": [170, 255, 0],
                            "strokeWidth": 0.5
                        },
                        "textLabel": [
                            {
                                "field": {
                                    "name": "route_name"
                                },
                                "color": [255, 255, 255],
                                "size": 18,
                                "offset": [0, 0],
                                "anchor": "start",
                                "alignment": "center"
                            }
                        ]
                    }
                }
            ]
        }
    }
}

# Display the map (this would work in a Jupyter notebook or similar environment)
m

In [ ]:
def add_edges2(G, gdf): #egyelőre nem kell
    for idx, row in gdf.iterrows():
        # Extract start and end coordinates
        start_coords = (row['geometry'].coords[0][1], row['geometry'].coords[0][0])
        end_coords = (row['geometry'].coords[-1][1], row['geometry'].coords[-1][0])

        # Create a unique key for the edge
        key = f"{row['route_name']}_{row['ntrips']}"

        # Add nodes with stop names
        G.add_node(start_coords, stop_name=row['start_stop_name'])
        G.add_node(end_coords, stop_name=row['end_stop_name'])

        # Add edge to the graph with specified keys
        G.add_edge(start_coords, end_coords, key=key, route_name=row['route_name'], ntrips=row['ntrips'])

In [ ]:
G3 = nx.MultiDiGraph()
add_edges2(G3, segments_freq)

In [ ]:
def add_edgesline(m, gdf):
    for idx, row in gdf.iterrows():
        if isinstance(row['geometry'], LineString):
            coords = [(pt[1], pt[0]) for pt in row['geometry'].coords]
            tooltip_text = f"Route: {row['route_name']}<br>Trips: {row['ntrips']}<br>From: {row['start_stop_name']}<br>To: {row['end_stop_name']}"
            folium.PolyLine(coords, color='blue', weight=2.5, opacity=0.8, tooltip=tooltip_text).add_to(m)

In [ ]:
m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)
add_edgesline(m, segments_freq)

# Save the map to an HTML file
#m.save('budapest_transit_map.html')

# Display the map
m

In [ ]:
lines = []
for idx, row in segments_freq.iterrows():
    if isinstance(row['geometry'], LineString):
        coords = list(row['geometry'].coords)
        lat, lon = zip(*coords)
        lines.append(go.Scattermapbox(
            lat=lat,
            lon=lon,
            mode='lines',
            line=dict(width=2, color='blue'),
            text=f"Route: {row['route_name']}<br>Trips: {row['ntrips']}<br>From: {row['start_stop_name']}<br>To: {row['end_stop_name']}",
            hoverinfo='text'
        ))

# Create a Plotly figure
fig = go.Figure()

fig.add_traces(lines)

fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_zoom=12,
    mapbox_center={"lat": 47.4979, "lon": 19.0402},
    height=600
)

fig.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
m = Basemap(projection='merc', llcrnrlat=47.0, urcrnrlat=48.0, llcrnrlon=18.5, urcrnrlon=19.5, resolution='i', ax=ax)
m.drawcoastlines()
m.drawcountries()
m.drawmapboundary(fill_color='aqua')
m.fillcontinents(color='lightgreen', lake_color='aqua')
m.drawparallels(range(47, 49), labels=[1,0,0,0])
m.drawmeridians(range(18, 20), labels=[0,0,0,1])

# Add segments to the map
for idx, row in segments_freq.iterrows():
    if isinstance(row['geometry'], LineString):
        coords = list(row['geometry'].coords)
        lons, lats = zip(*coords)
        x, y = m(lons, lats)
        m.plot(x, y, marker=None, color='b', linewidth=2, label=f"{row['route_name']} ({row['ntrips']} trips)")

# Add a legend
plt.legend(loc='lower left', fontsize='small')

plt.title('Budapest Public Transport Segments')
plt.show()